# Soilgrids Analysis
## Water Storage Capacity Analysis for Bachelor Thesis

This notebook contains the main code for analyzing water storage capacity using soilgrids data.

## 1. Import Libraries

In [3]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import soilgrids
from soilgrids import SoilGrids
import rasterio
import xarray as xr
import pyproj

# Add more libraries as needed for soilgrids analysis

In [7]:
#initialize SoilGrids 
sg = SoilGrids()


In [8]:
sg.get_coverage_list("silt")

"silt" map service includes 30 coverages(maps)

silt_0-5cm_Q0.05
silt_0-5cm_Q0.5
silt_0-5cm_Q0.95
silt_0-5cm_mean
silt_0-5cm_uncertainty
silt_5-15cm_Q0.05
silt_5-15cm_Q0.5
silt_5-15cm_Q0.95
silt_5-15cm_mean
silt_5-15cm_uncertainty
silt_15-30cm_Q0.05
silt_15-30cm_Q0.5
silt_15-30cm_Q0.95
silt_15-30cm_mean
silt_15-30cm_uncertainty
silt_30-60cm_Q0.5
silt_30-60cm_Q0.05
silt_30-60cm_Q0.95
silt_30-60cm_mean
silt_30-60cm_uncertainty
silt_60-100cm_Q0.05
silt_60-100cm_Q0.5
silt_60-100cm_Q0.95
silt_60-100cm_mean
silt_60-100cm_uncertainty
silt_100-200cm_Q0.5
silt_100-200cm_Q0.05
silt_100-200cm_Q0.95
silt_100-200cm_mean
silt_100-200cm_uncertainty


In [ ]:
#TODO: 
# get bounding box
# get data from bounding box
# get formula for litterally anything just to test it out
# apply data to formula
# visualize data

# required variables 
- sand  # sand content [%]
- clay  # clay content [%]
- soc   # soil organic carbon [g kg-1]
- bdod    # bulk density [kg m-3]

In [9]:
def calculate_dimensions(west, south, east, north, crs, pixel_size_m=250):
    """
    Calculate width and height in pixels based on bounding box and desired pixel size.

    Parameters:
    -----------
    west, south, east, north : float
        Bounding box coordinates in the specified CRS
    crs : str
        Coordinate reference system (e.g., "EPSG:4326")
    pixel_size_m : float, optional
        Desired pixel size in meters (default: 250)

    Returns:
    --------
    tuple : (width, height) in pixels
    """
    # Create transformer to convert from lat/lon to meters
    # Use a local UTM or equidistant projection for accurate distance calculation

    # Get the center point to determine appropriate UTM zone
    center_lon = (west + east) / 2
    center_lat = (south + north) / 2

    # Create a local equal-area projection centered on the AOI
    proj_string = f"+proj=aeqd +lat_0={center_lat} +lon_0={center_lon} +x_0=0 +y_0=0 +datum=WGS84 +units=m +no_defs"

    # Create coordinate transformer
    transformer = pyproj.Transformer.from_crs(
        crs,
        proj_string,
        always_xy=True
    )

    # Transform corner coordinates to meters
    west_m, south_m = transformer.transform(west, south)
    east_m, north_m = transformer.transform(east, north)

    # Calculate dimensions in meters
    width_m = east_m - west_m
    height_m = north_m - south_m

    # Calculate dimensions in pixels
    width_px = int(abs(width_m) / pixel_size_m)
    height_px = int(abs(height_m) / pixel_size_m)

    return width_px, height_px

In [10]:
# define the crs and bounding box
crs = "urn:ogc:def:crs:EPSG::4326"
west, south, east, north = [-79.30, -2.87, -79.18, -2.77]
width, height = calculate_dimensions(west, south, east, north, crs, pixel_size_m=250)
print(f"Width: {width} pixels")
print(f"Height: {height} pixels")

Width: 53 pixels
Height: 44 pixels


In [ ]:
#option 2: use resolutions instead of dimensions
import math
lat = (south + north) / 2
meters_per_degree_lon = 111320 * math.cos(math.radians(abs(lat)))
meters_per_degree_lat = 111320

resx = 250 / meters_per_degree_lon
resy = 250 / meters_per_degree_lat

## Download Soil Properties Data (for all depths)
Download all required soil properties from SoilGrids for the specified bounding box.

In [ ]:
# Define the soil properties and depth layers to download
soil_properties = {
    'sand': 'sand',
    'clay': 'clay',
    'silt': 'silt',
    'soc': 'soc',
    'bdod': 'bdod'
}

# Define depth layers
depth_layers = ['0-5cm', '5-15cm', '15-30cm', '30-60cm', '60-100cm', '100-200cm']

# Dictionary to store the downloaded data
soil_data = {}

# Download each soil property for each depth layer
for depth in depth_layers:
    print(f"\n{'='*60}")
    print(f"Processing depth layer: {depth}")
    print(f"{'='*60}")
    
    soil_data[depth] = {}
    
    for property_name in soil_properties.keys():
        coverage_id = f'{property_name}_{depth}_mean'
        print(f"\nDownloading {property_name} for {depth}...")
        
        try:
            data = sg.get_coverage_data(
                service_id=property_name,
                coverage_id=coverage_id,
                crs=crs,
                west=west,
                south=south,
                east=east,
                north=north,
                width=width,
                height=height,
                output=f'output_data/{property_name}/{property_name}_{depth}.tif'
            )
            
            # Mask NoData values
            nodata = data.attrs.get('_FillValue', -32768)
            data_masked = data.where(data != nodata)
            
            # Store in dictionary
            soil_data[depth][property_name] = data_masked
            
            # Print basic statistics
            print(f"{property_name} ({depth}) - Min: {data_masked.min().values:.2f}, "
                  f"Max: {data_masked.max().values:.2f}, "
                  f"Mean: {data_masked.mean().values:.2f}")
        
        except Exception as e:
            print(f"Error downloading {property_name} for {depth}: {e}")
            soil_data[depth][property_name] = None

print("\n" + "="*60)
print("✓ All soil properties downloaded successfully!")
print("="*60)


sand - Min: 372.00, Max: 484.00, Mean: 422.96

clay - Min: 141.00, Max: 248.00, Mean: 190.88

silt - Min: 323.00, Max: 429.00, Mean: 386.16

soc - Min: 558.00, Max: 1931.00, Mean: 1215.48

bdod - Min: 92.00, Max: 118.00, Mean: 104.99

✓ All soil properties downloaded successfully!


In [9]:
# Access individual variables for convenience (0-5cm depth)
sand_0_5cm = soil_data['0-5cm']['sand']
clay_0_5cm = soil_data['0-5cm']['clay']
silt_0_5cm = soil_data['0-5cm']['silt']
soc_0_5cm = soil_data['0-5cm']['soc']
bdod_0_5cm = soil_data['0-5cm']['bdod']

print("Variables created for 0-5cm depth:")
print("- sand_0_5cm: Sand content [g/kg]")
print("- clay_0_5cm: Clay content [g/kg]")
print("- silt_0_5cm: Silt content [g/kg]")
print("- soc_0_5cm: Soil organic carbon [dg/kg]")
print("- bdod_0_5cm: Bulk density [cg/cm³]")
print("\nAll data also accessible via soil_data dictionary:")
print("Example: soil_data['5-15cm']['sand']")

NameError: name 'soil_data' is not defined

## 3. Data Processing and Analysis

In [ ]:
# lets get the awc_layer = fc_layer - pwp_layer
# fc_layer is the wv0033 data
# pwp_layer is the wv1500 data
# use the data from the folder water_cap and calculate the awc for each depth

## 4. Visualization

In [ ]:
# Create visualizations
# Example:
# plt.figure(figsize=(10, 6))
# plt.plot(data)
# plt.title('Water Storage Capacity Analysis')
# plt.xlabel('X-axis')
# plt.ylabel('Y-axis')
# plt.show()

## 5. Save Results

In [ ]:
# Save results to output_data folder
# Example:
# results.to_csv('output_data/analysis_results.csv', index=False)